In [ ]:
# !pip install wget
# !pip install tqdm

# Importing Necessary Dependencies

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import RandomHorizontalFlip, Resize, ToTensor, Compose, RandomRotation, RandomResizedCrop, RandomVerticalFlip
import os
from PIL import Image
torch.manual_seed(50)
random.seed(50)
np.random.seed(50)

# Check if Cuda is available or not

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Wandb login

In [ ]:
import wandb
!wandb login f6b40fc0bcc13c1d6117718b14ef1ddd1c68a700

# Implementing Convolutional Neural Network and Batch Normalization Implemented 

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_filters, size_filters, activation_function, dense_neurons, img_size, get_paramters, batch_normalization, dropout_prob, padding = 0, stride = 1, padding_pool = 0, size_filter_pool = 2, stride_pooling = 2):
        self.batch_normalization = batch_normalization
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, num_filters[0], size_filters[0])
        if(self.batch_normalization):
            self.bn1 = nn.BatchNorm2d(num_filters[0])  # BatchNorm after first Conv2d
        self.conv2 = nn.Conv2d(num_filters[0], num_filters[1], size_filters[1])
        if(self.batch_normalization):
            self.bn2 = nn.BatchNorm2d(num_filters[1])  # BatchNorm after second Conv2d
        self.conv3 = nn.Conv2d(num_filters[1], num_filters[2], size_filters[2])
        if(self.batch_normalization):
            self.bn3 = nn.BatchNorm2d(num_filters[2])  # BatchNorm after third Conv2d
        self.conv4 = nn.Conv2d(num_filters[2], num_filters[3], size_filters[3])
        if(self.batch_normalization):
            self.bn4 = nn.BatchNorm2d(num_filters[3])  # BatchNorm after fourth Conv2d
        self.conv5 = nn.Conv2d(num_filters[3], num_filters[4], size_filters[4])
        if(self.batch_normalization):
            self.bn5 = nn.BatchNorm2d(num_filters[4])  # BatchNorm after fifth Conv2d

        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout_prob)
        self.pool = nn.MaxPool2d(2, 2)
        self.activation = nn.ReLU()  # Default activation is ReLU
        
        if activation_function.lower() == 'relu':
            self.activation = nn.ReLU()
        elif activation_function.lower() == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation_function.lower() == 'tanh':
            self.activation = nn.Tanh()
        elif activation_function.lower() == 'leaky_relu':
            self.activation = nn.LeakyReLU()
        elif activation_function.lower() == 'gelu':
            self.activation = nn.GELU()
        elif activation_function.lower() == 'silu':
            self.activation = nn.SiLU()
        elif activation_function.lower() == 'mish':
            self.activation = Mish()
        
        
        image_size = img_size
        for it in range(5):
            image_size = self.update_image_size(image_size, size_filters[it], padding, stride, padding_pool, size_filter_pool, stride_pooling)
            print(image_size)                   
        
        

        self.fc1 = nn.Linear(num_filters[4]*image_size*image_size, dense_neurons)
        
        self.output_layer = nn.Linear(dense_neurons, 10)
    
    def update_image_size(self, w, f, p, s, pp, fp, sp):
        res = (w - f + 2*p) // s + 1
        res = (res - fp  + 2*pp) // sp + 1
        return res
    
    def forward(self, x):
        
        if(self.batch_normalization):
            x = self.pool(self.activation(self.bn1(self.conv1(x))))  # BatchNorm after first Conv2d
            x = self.pool(self.activation(self.bn2(self.conv2(x))))  # BatchNorm after second Conv2d
            x = self.pool(self.activation(self.bn3(self.conv3(x))))  # BatchNorm after third Conv2d
            x = self.pool(self.activation(self.bn4(self.conv4(x))))  # BatchNorm after fourth Conv2d
            x = self.pool(self.activation(self.bn5(self.conv5(x))))  # BatchNorm after fifth Conv2d

        else:
            x = self.pool(self.activation(self.conv1(x)))
            x = self.pool(self.activation(self.conv2(x)))
            x = self.pool(self.activation(self.conv3(x)))
            x = self.pool(self.activation(self.conv4(x)))
            x = self.pool(self.activation(self.conv5(x)))
            
        x = self.flatten(x)                                 # Flatten for fully connected layers
        x = self.dropout(x)
        x = self.fc1(x)
        x = F.softmax(self.output_layer(x),dim=1)
        # print("code added for checking")
        return x


# Training model Function
## - Data Augmentation implemented 
## - Train set is splited into Val and Train
 


In [ ]:
def train_model(data_aug, wandLog, model, train_dir, val_dir, num_epochs, batch_size, learning_rate, optimizer_name, img_size):
    # Define transforms for data augmentation and normalization
    if data_aug:
        transform_train = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            RandomRotation(10),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
#             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    else:
        transform_train = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
#             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    transform_val = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        # transforms.CenterCrop(32),
        transforms.ToTensor(),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load the datasets using ImageFolder
    train_dataset = ImageFolder(train_dir, transform=transform_train)
    val_dataset = ImageFolder(val_dir, transform=transform_val)


    labels = train_dataset.classes
    train_set, val_set = random_split(train_dataset, [8000, 1999])

    # Create data loaders for training and validation
    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers = 4,shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size,num_workers = 4, shuffle=True)
    test_loader = DataLoader(val_dataset, batch_size=batch_size,num_workers = 4, shuffle=True)

    criterion = nn.CrossEntropyLoss()
    if optimizer_name.lower() == 'sgd':
        # print("SGD")
        optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9)
    elif optimizer_name.lower() == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr = learning_rate, alpha = 0.99, eps = 1e-8)
        # print("RMSPROP")
    elif optimizer_name.lower() == 'adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr = learning_rate, lr_decay = 0, weight_decay = 0, initial_accumulator_value = 0, eps = 1e-10)
        # print("ADAGRAD")
    else:
        optimizer = optim.Adam(model.parameters(), lr = learning_rate, betas = (0.9, 0.999), eps = 1e-8)
        # print("ADAM")
    
    
    epoch = 0
    while epoch < num_epochs:
        model.train()  # Set model to training mode
        # count = 0
        running_loss, train_correct_p, train_total_p = 0.0, 0, 0
        
        for i, data in train_loader:
            # print(i)
            inputs = i.to(device)
            labels = data.to(device)
#             inputs, labels = i, data
            
            optimizer.zero_grad()

            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss = running_loss + loss.item()
            _, pred = torch.max(outputs.data, 1)
            train_total_p += labels.size(0)
            train_correct_p += (pred == labels).sum().item()
            
        
        # count += 1
#         print(count)
        
        # Validate the model after each epoch
        
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for i, data in val_loader:
                inputs, labels = i.to(device), data.to(device)
#                 inputs, labels = i, data
                outputs = model.forward(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()


        # Print Train Statistics
        running_loss = running_loss / len(train_loader)
        acur_on_scale_1 = train_correct_p / train_total_p
        train_accuracy = acur_on_scale_1 * 100
        print(f'Epoch {epoch+1}, Train Loss: {running_loss:.3f}, Train Accuracy: {train_accuracy:.2f}%')      
        # Print validation statistics
        val_loss = val_loss/len(val_loader)
        temp2 = correct / total
        val_accuracy = 100 * temp2
        print(f'Epoch {epoch + 1}, Validation Loss: {val_loss:.3f}, Validation Accuracy: {val_accuracy:.2f}%')
        print()
        if wandLog:
            wandb.log(
                {
                    'epoch': epoch+1,
                    'training_loss' : running_loss,
                    'training_accuracy' : train_accuracy,
                    'validation_loss' : val_loss,
                    'validation_accuracy':val_accuracy
                }
            )
        epoch += 1
    print('Training finished')
    return model, val_accuracy 





# Sweep Configuration

In [ ]:
sweep_config = {
            'name': 'sweep 1 : bayes',
            'method': 'bayes',
            'metric': { 'goal': 'maximize','name': 'Accuracy'},
            'parameters': 
                {
                    'epochs' : {'values': [25]},
                    'num_kernels' : {'values': [[16, 32, 64, 128, 256], [16, 16, 32, 32, 16], [16, 32, 16, 32, 16], [32, 32, 32, 32, 32]]},
                    'kernel_sizes' : {'values': [[11, 9, 7, 5, 3], [11, 9, 7, 7, 5], [7, 7, 5, 5, 3], [11, 11, 7, 7, 3]]},
                    'batch_size' : {'values': [32]},
                    'dense_sizes' : {'values': [256]},
                    'activation' : {'values': ['ReLU', 'LeakyReLU']}, 
                    'optimizer' : {'values' : ['adam']},
                    'learning_rate' : {'values': [0.0001, 0.0003]},
                    'batch_norm' : {'values': [True]},
                    'dropout' : {'values': [0.2, 0.3]},
                    'img_size' : {'values': [256, 224]},
                    'data_aug' : {'values' : [False]}
                }
            }

In [ ]:

def run_sweep():
    init = wandb.init(project = 'Deep Learning Assignment 2 Random Sweep')
    config = init.config
    
    batch_size = config.batch_size
    num_epochs = config.epochs
    learning_rate = config.learning_rate
    optimizer_name = config.optimizer
    dropout_prob = config.dropout
    batch_normalization = config.batch_norm
    img_size = config.img_size
    dense_neurons = config.dense_sizes
    activation_function = config.activation
    wandLog = 1
    num_filters = config.num_kernels
    size_filters = config.kernel_sizes
    get_paramters = True
    data_aug = config.data_aug
    
    train_dir = '/kaggle/input/nature/inaturalist_12K/train'
    val_dir = '/kaggle/input/nature/inaturalist_12K/val'
    
    wandb.run.name = (
        'ACTIV_:' + activation_function +
        'OP_:' + optimizer_name +
        'SF_:' + ','.join(map(str, size_filters)) +
        'NF_:' + str(num_filters) +
        'IMGSI_:' + str(img_size)+
        'LR_:' + str(learning_rate) +
        'E_:' + str(num_epochs) +
        'DN_:' + str(dense_neurons) +
        'BATCHS_:' + str(batch_size) +
        'BATCHN_:' + str(batch_normalization) +
        'ProbD_:' + str(dropout_prob) +
        'AUG_:' + str(data_aug)
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = ConvNet(num_filters, size_filters, activation_function, dense_neurons, img_size, get_paramters, batch_normalization, dropout_prob).to(device)
#     model = ConvNet(num_filters, size_filters, activation_function, dense_neurons, img_size, get_paramters, batch_normalization, dropout_prob)
    
    model_after_train, v_a = train_model(data_aug, wandLog, model, train_dir, val_dir, num_epochs, batch_size, learning_rate, optimizer_name, img_size)      
    wandb.log({'Accuracy': v_a})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='Deep Learning Assignment 2 Random Sweep')
wandb.agent(sweep_id, run_sweep, count = 30)
wandb.finish()